In [78]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder 


In [83]:
df = pd.read_csv('data_for_test.csv')

In [84]:
labelencoder = LabelEncoder()
df.creator = labelencoder.fit_transform(df.creator)

labelencodergen = LabelEncoder()
df.gender = labelencodergen.fit_transform(df.gender)

labelencoderrace = LabelEncoder()
df.type_race = labelencodergen.fit_transform(df.type_race)

In [85]:
df.drop(columns=["overall_score","name","real_name","full_name","height","weight"],inplace=True)

In [86]:
df.dropna(inplace=True)

In [87]:
df

,history_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,creator,gender,type_race
0,"Delroy Garrett, Jr. grew up to become a track ...",85,30,60,60,40,70,22,1,27
1,He was one of the many prisoners of Indian Hil...,100,20,30,50,35,100,5,2,67
2,"Richard ""Rick"" Jones was orphaned at a young ...",80,100,80,100,100,80,22,1,27
3,Aa is one of the more passive members of the P...,80,50,55,45,100,55,5,1,27
4,Aaron Cash is the head of security at Arkham A...,80,10,25,40,30,50,5,1,27
...,...,...,...,...,...,...,...,...,...,...
1445,Zatanna is the daughter of adventurer John Zat...,90,10,25,30,100,55,5,0,27
1446,Zero was created by the late Dr. Albert Wily ...,80,100,100,100,100,80,2,1,51
1447,"Hunter Zolomon is better known as Zoom, a spee...",95,50,100,75,100,80,5,1,67
1448,Hunter Zolomon had a troubled relationship wi...,75,10,100,30,100,30,5,1,67


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1360 entries, 0 to 1449
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   history_text        1360 non-null   object
 1   intelligence_score  1360 non-null   int64 
 2   strength_score      1360 non-null   int64 
 3   speed_score         1360 non-null   int64 
 4   durability_score    1360 non-null   int64 
 5   power_score         1360 non-null   int64 
 6   combat_score        1360 non-null   int64 
 7   creator             1360 non-null   int32 
 8   gender              1360 non-null   int32 
 9   type_race           1360 non-null   int32 
dtypes: int32(3), int64(6), object(1)
memory usage: 100.9+ KB


## Splitting 

In [102]:
test = df.iloc[:200]
train = df.iloc[200:]

##  train without Text

In [103]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(train.drop(['history_text','creator'],axis=1), train["creator"])


C:\Users\dada\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [ ]:
y_pred = model.predict(test.drop(['history_text','creator'],axis=1))

acc = accuracy_score(test["creator"], y_pred)

print("Accuracy : {:.2f}".format(acc*100))

##  train withText

In [106]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(df['history_text'])


CountVectorizer()

In [107]:
train1 = vectorizer.transform(train['history_text'])
test1 = vectorizer.transform(test['history_text'])

In [108]:
model = LinearSVC()
model.fit(train1, train["creator"])

C:\Users\dada\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [109]:
y_pred = model.predict(test1)

acc = accuracy_score(test["creator"], y_pred)

print("Accuracy : {:.2f}".format(acc*100))

Accuracy : 73.00


##  with both

on va essayer de concatener les features extrait du history text avec les caractéristique de chaque héro pour voir si ça donne de meilleur résultats

In [129]:
tr = np.concatenate((train1.toarray(), train.drop(['history_text','creator'],axis=1).to_numpy()), axis=1)
ts = np.concatenate((test1.toarray(), test.drop(['history_text','creator'],axis=1).to_numpy()), axis=1)

In [130]:
model = LinearSVC()
model.fit(tr, train["creator"])

C:\Users\dada\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [131]:
y_pred = model.predict(ts)

acc = accuracy_score(test["creator"], y_pred)

print("Accuracy : {:.2f}".format(acc*100))

Accuracy : 67.00
